In [1]:
from dotenv import load_dotenv
import os
import openai
from datasets import load_dataset
import llama_index
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import PineconeVectorStore
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
import chromadb
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.prompts import PromptTemplate
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.node_parser import SimpleNodeParser
from pathlib import Path
from llama_index import download_loader
from llama_index.embeddings import OpenAIEmbedding
from llama_index.text_splitter import SentenceSplitter
from llama_index.extractors import TitleExtractor
from llama_index.ingestion import IngestionPipeline, IngestionCache
from llama_index import download_loader
from pathlib import Path
from llama_index import download_loader
from llama_index import VectorStoreIndex
from llama_index import SummaryIndex
import openai
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    get_response_synthesizer,
)
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)

from llama_index.ingestion import IngestionPipeline
from llama_index.text_splitter import TokenTextSplitter

from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [2]:
OPENAI_API_KEY=
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
storage_context = StorageContext.from_defaults(
    docstore= SimpleDocumentStore.from_persist_dir(persist_dir="C:/Users/zachk/OneDrive/Desktop/Uflo/Indexes"),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="C:/Users/zachk/OneDrive/Desktop/Uflo/Indexes")
)

ds384 = load_index_from_storage(storage_context)

In [4]:
PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path("C:/Users/zachk/Downloads/(13a) Civil War Reconstruction  (1).pdf"))

In [5]:
pages = documents[1:3]

## BEGIN TO MAKE FLASHCARD

In [6]:
def get_llm(openai_api_key, max_tokens=8192):
    os.environ["OPENAI_API_KEY"] = openai_api_key
    return OpenAI(
        temperature=0.0, model='gpt-3.5-turbo-16k', max_tokens=max_tokens
    )

In [7]:
llm = get_llm(OPENAI_API_KEY)


In [8]:
def get_ds_index(documents, llm, api_key):

    service_context = ServiceContext.from_defaults(llm=llm, chunk_size=384, chunk_overlap=128)

    response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
    )
    
    temp_index = DocumentSummaryIndex.from_documents(
        documents, 
        service_context=service_context,
        response_synthesizer = response_synthesizer
    )

    return temp_index

In [9]:
ds384 = get_ds_index(pages, llm, OPENAI_API_KEY)

current doc id: f27b63bf-8f9c-45bc-a85a-9a85aa4f14b1
current doc id: b1b35489-93e0-43c0-abf1-0c3d5f5d77a1


In [10]:
def extract_terms_1(index, prompt, summary, k=1):
    query_engine = index.as_query_engine(
        similarity_top_k=k
    )

    f_prompt = prompt.format(summary)

    terms_definitions = str(query_engine.query(f_prompt))
    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }
    return terms_to_definition

In [11]:
def summarize_docs(index, prompt, k=1):
    query_engine = index.as_query_engine(similarity_top_k=k)

    summary = str(query_engine.query(prompt))

    return summary


In [12]:
def extract_terms(index, summary, prompts, k=1):
    query_engine = index.as_query_engine(similarity_top_k=k)

    all_terms_to_definition = {}
    for prompt in prompts:
        terms_definitions = str(query_engine.query(prompt))
        terms_definitions = [
            x
            for x in terms_definitions.split("\n")
            if x and "Term:" in x and "Definition:" in x
        ]
        for term_definition in terms_definitions:
            term = term_definition.split("Definition:")[0].split("Term:")[-1].strip()
            definition = term_definition.split("Definition:")[-1].strip()

            if term in all_terms_to_definition:
                # Combine definitions if they are different
                if all_terms_to_definition[term] != definition:
                    all_terms_to_definition[term] += " Additionally, " + definition
            else:
                all_terms_to_definition[term] = definition

    return all_terms_to_definition

In [13]:
def check_terms(index, context_str, prompt, k = 1):
    query_engine = index.as_query_engine(similarity_top_k = k)

    context_str = str(context_str)

    final_prompt = prompt.format(context_str)

    terms_definitions = str(query_engine.query(final_prompt))
    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }
    return terms_to_definition

    

In [146]:
summaryPrompt = (
    """Task: Carefully read the document and produce a concise summary. 
    It is crucial to highlight all main themes, crucial events, key figures, and significant concepts, as missing any could have dire consequences.
    ensure that you mention all people and events of importance that had any significant context in the field.
    Ensure the summary distills the document's content into its most essential elements, capturing every potential term and idea without omission. 
    The accuracy and comprehensiveness of this summary are vital, as if a life depends on it, setting the stage for accurate term and definition extraction in the next phase."""
)

summaryPrompt2 = (
    """Task: Conduct a thorough analysis of the text in three distinct steps. 
    First, meticulously identify and list every human name mentioned, focusing on both historical figures and any other individuals. 
    Second, enumerate all events detailed in the text. 
    Third, extract all significant terms. 
    Following these steps, compile this information into a detailed summary. 
    Ensure the summary coherently incorporates all names, events, and terms, offering a clear and complete overview of the text's content, with an emphasis on ensuring no human names are omitted."""
)
summaryPrompt3 = (
    """Task: Conduct a detailed analysis of the text in these specific steps:
    1. Meticulously identify and list every human name mentioned, focusing on both historical figures and any other individuals. Pay special attention to ensure no name is overlooked.
    2. Carefully extract and enumerate all events detailed in the text, including both major and minor occurrences, ensuring a comprehensive capture of the timeline.
    3. Thoroughly identify all significant terms and concepts, including technical jargon, theories, or specific subject matter terminology.
    4. Make sure to get as many terms as possible in this summary 
    After completing these steps, synthesize this information into a cohesive and comprehensive summary. The summary should seamlessly incorporate the identified names, events, and terms, providing a clear, accurate, and complete overview of the text's content."""
)


In [147]:
s1 = summarize_docs(ds384, summaryPrompt3,5)

In [148]:
s1

"1. Human names mentioned:\n- President Lincoln\n- Mathew Brady\n\n2. Events detailed in the text:\n- The Civil War\n- President Lincoln's leadership during the war\n- President Lincoln's decision on emancipation\n- The casualties and devastation of the war\n- The industrial boom in the North after the war\n- The slow and tortured rebuilding of the South\n- The surrender of the Confederate armies to Union troops\n\n3. Significant terms and concepts:\n- Civil War\n- Emancipation\n- Modern warfare\n- Gilded Age\n- Land Grant College Act\n- Protective tariff\n- Industrialization\n- Global economic power\n- Union blockade\n- Defensive war\n\nSummary: The text discusses the impact of the Civil War in the United States. It highlights President Lincoln's leadership during the war and his decision on emancipation. The war resulted in a high number of casualties and devastation, ushering in the reality of modern warfare. The North experienced an industrial boom and emerged as a global economic 

In [67]:
prompts = [term_prompt1,combinedPrompt]

In [213]:
t1 = extract_terms_1(ds384, term_prompt1, s1, 5)

In [214]:
t1

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist who documented the carnage of the Civil War through photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': 'The act of freeing slaves from bondage.',
 'Modern warfare': 'The use of advanced technology and tactics in warfare, resulting in high casualties and devastation.',
 'Gilded Age': 'A period of economic growth and industrialization in the United States after the Civil War, characterized by wealth and corruption.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided funding for the establishment of universities focused on agriculture and mechanical arts.',
 'Protective tariff': 'A tax imposed on imported goods to protect domestic industries from foreign competition.',
 'Industrialization': 'The process of developing industries

In [176]:
t1

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist who documented the carnage of the Civil War through photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': 'The act of freeing slaves from bondage.',
 'Modern warfare': 'The use of advanced technology and tactics in warfare, resulting in high casualties and devastation.',
 'Gilded Age': 'A period of economic growth and industrialization in the United States after the Civil War, characterized by wealth and corruption.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided funding for the establishment of universities focused on agriculture and mechanical arts.',
 'Protective tariff': 'A tax imposed on imported goods to protect domestic industries from foreign competition.',
 'Industrialization': 'The process of developing industries

In [154]:
t1

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist who documented the carnage of the Civil War through photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': 'The act of freeing slaves from bondage.',
 'Modern warfare': 'The use of advanced technology and tactics in warfare, resulting in high casualties and devastation.',
 'Gilded Age': 'A period of economic growth and industrialization in the United States after the Civil War, characterized by wealth and corruption.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided funding for the establishment of universities focused on agriculture and mechanical arts.',
 'Protective tariff': 'A tax imposed on imported goods to protect domestic industries from foreign competition.',
 'Industrialization': 'The process of developing industries

In [31]:
t1

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist and photographer who documented the carnage of the Civil War through his photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': 'The act of freeing enslaved individuals, which President Lincoln decided to do during the Civil War.',
 'Modern warfare': 'The harsh reality of warfare during the Civil War, which included high casualties and widespread devastation.',
 'Gilded Age': 'A term used to describe the era following the Civil War, characterized by superficial glitter and beauty covering up underlying decay.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided money from the sale of public lands to states for the establishment of universities that taught agriculture and mechanical arts.',
 'Protective tariff': 'Legislation passe

In [104]:
c2

{'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Confederacy': 'The group of Southern states that seceded from the United States and formed their own government during the Civil War.',
 'Union troops': 'The military forces of the Northern states during the Civil War.',
 'Gettysburg': 'A major battle fought in Gettysburg, Pennsylvania, in July 1863, considered a turning point in the Civil War.',
 'Petersburg': 'A city in Virginia where a significant battle took place during the Civil War, known as the Siege of Petersburg.',
 'Atlanta': 'A city in Georgia that was a major target for Union forces during the Civil War, and was eventually captured in September 1864.',
 'Appomattox Court House': 'The location in Virginia where Confederate General Robert E. Lee surrendered to Union General Ulysses S. Grant, effectively ending the Civil War.',
 'Defensive war': 'A war strategy in which a side prima

In [79]:
d14= extract_terms(ds384, prompt_list, 2)  # Here, 3 is the similarity_top_k value

In [215]:
c3 = check_terms(ds384, t1, finalCheckPrompt,5)

In [216]:
c3

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist who documented the carnage of the Civil War through photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': 'The act of freeing slaves from bondage.',
 'Modern warfare': 'The use of advanced technology and tactics in warfare, resulting in high casualties and devastation.',
 'Gilded Age': 'A period of economic growth and industrialization in the United States after the Civil War, characterized by wealth and corruption.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided funding for the establishment of universities focused on agriculture and mechanical arts.',
 'Protective tariff': 'A tax imposed on imported goods to protect domestic industries from foreign competition.',
 'Industrialization': 'The process of developing industries

In [133]:
c3

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist and photographer who documented the carnage of the Civil War through his photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': "The act of freeing enslaved individuals, referring to President Lincoln's decision to issue the Emancipation Proclamation during the Civil War.",
 'Modern warfare': 'The harsh reality of warfare during the Civil War, characterized by high casualties and the use of new technologies.',
 'Gilded Age': 'A term used to describe the era following the Civil War, characterized by superficial glitter and beauty covering up underlying decay.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided money from the sale of public lands to states for the establishment of universities focused on agriculture and mechanical

In [131]:
c2

{'President Lincoln': 'The leader of the United States during the Civil War.',
 'Mathew Brady': 'A journalist and photographer who documented the carnage of the Civil War through his photographs.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Emancipation': "The act of freeing enslaved individuals, referring to President Lincoln's decision to issue the Emancipation Proclamation during the Civil War.",
 'Modern warfare': 'The harsh reality of warfare during the Civil War, characterized by high casualties and the use of new technologies.',
 'Gilded Age': 'A term used to describe the era following the Civil War, characterized by superficial glitter and beauty covering up underlying decay.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided money from the sale of public lands to states for the establishment of universities focused on agriculture and mechanical

In [80]:
d14

{'Union': 'A side in the Civil War that fought to preserve the United States and abolish slavery.',
 'Confederate': 'A side in the Civil War that fought for the independence of the Southern states and the preservation of slavery.',
 'Union forces': 'The military forces of the Union side in the Civil War.',
 'Confederate forces': 'The military forces of the Confederate side in the Civil War.',
 'Union victory': 'A successful outcome for the Union side in a battle or the overall war.',
 'Confederate victory': 'A successful outcome for the Confederate side in a battle or the overall war.',
 'Gettysburg': 'A town in Pennsylvania where a major battle of the Civil War took place in 1863.',
 'Petersburg': 'A city in Virginia where a major battle of the Civil War took place in 1864-1865.',
 'Atlanta': 'A city in Georgia that was a key target for the Union forces during the Civil War.',
 'Appomattox Court House': 'A village in Virginia where General Robert E. Lee surrendered to General Ulysses 

In [82]:
c1

{'Union': 'A side in the Civil War that fought to preserve the United States and abolish slavery.',
 'Confederate': 'A side in the Civil War that fought for the independence of the Southern states and the preservation of slavery.',
 'Union forces': 'The military forces of the Union side in the Civil War.',
 'Confederate forces': 'The military forces of the Confederate side in the Civil War.',
 'Union victory': 'A successful outcome for the Union side in a battle or the overall war.',
 'Confederate victory': 'A successful outcome for the Confederate side in a battle or the overall war.',
 'Gettysburg': 'A town in Pennsylvania where a major battle of the Civil War took place in 1863.',
 'Petersburg': 'A city in Virginia where a major battle of the Civil War took place in 1864-1865.',
 'Atlanta': 'A city in Georgia that was a key target for the Union forces during the Civil War.',
 'Appomattox Court House': 'A village in Virginia where General Robert E. Lee surrendered to General Ulysses 

In [195]:
term_prompt = (
    "Make a list of any possible terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "A summary is provided in the last part of this prompt understand the prompt the use the summary to execute"
    "Analyze the provided summary and extract key terms and definitions. "
    "Identify and list every critical concept, entity, and technical term from the summary. "
    "For each identified term, write a precise and contextually appropriate definition. "
    "Ensure comprehensive coverage, including minor but relevant terms. "
    "Your objective is to gather a broad array of terms, providing a foundation for the detailed refinement in the next step. "
    "Response format:\nTerm: <term> Definition: <definition> "
    "Begin the term extraction from the summary"
    "{}"
    
)
term_prompt1 = (
    "Make a list of any terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "Your main focus/goal is to prioritize breadth over precision at this stage, aiming to capture as many terms as possible, regardless of their immediate perceived significance."

    "Write each line as as follows:\nTerm: <term> Definition: <definition>"

    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
)
term_prompt3 =(
    
    "Make a list of any terms and definitions that are defined in the context,and make sure to get as many terms as possible "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "Your main focus/goal is to prioritize breadth over precision at this stage, aiming to capture as many terms as possible, regardless of their immediate perceived significance."
    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"

    
)


term_prompt2 = (
    "Extract every term that you feel is import"
    "Define said term in context to the best of your ability"
    "with one paid on each line"
    "Get as many terms as possible as my life depends on having a large choice to pick from for the next step"

    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
)



In [208]:
prompt8 = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line."
    "there will be a name 'Mathew Brady' this needs to be in the dictionary or i die"
    "Note that formal names, like 'Mathew Brady', 'Abraham Lincoln' or 'Michael Duke', should be terms and should be given a definition. "
    "If a term is missing its definition, use your best judgment. "
    "With each term-definition pair on a separate line, be vigilant to not miss any names, regardless of their perceived prominence or importance. "
    "This task is crucial, as if my life depends on it, so please ensure the utmost accuracy and relevance. "
    "Write each line as follows:\nTerm: <term> Definition: <definition>"

    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
)
prompt2 = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
)


In [114]:
finalCheckPrompt = (
    """
    Task: Conduct a meticulous final review of the provided dictionary of terms represented by this string: {}.
    1. Ensure each term directly relates to the document's main subject.
    2. Eliminate redundant terms or definitions, keeping only distinct and crucial entries.
    3. Rigorously check for overlooked significant terms or concepts, adding them as necessary.
    4. If a definition mentions a term not yet listed (e.g., 'Confederacy' mentioned in the definition of 'Civil War'), include that term with its definition.
    5. Cross-check each definition for accuracy and relevance to the context.
    Your goal is to produce a refined, comprehensive list of terms and definitions, pertinent and devoid of repetition. Pay close attention to detail for the integrity and completeness of the final output.
    Response format:\nTerm: <term> Definition: <definition>

    Initiate the final refinement process:"""
)

finalCheckPrompt2 = (
    """
    With the following string which contains a text representation of a dictionary of terms, this is the string {}.
    Review each term to ensure it is directly relevant to the document's subject matter. 
    Remove any redundant terms or definitions, keeping only unique and necessary information. 
    Thoroughly check to see if any significant terms or concepts have been missed and add them if necessary.
    Revisit group mentions or frequently discussed topics to confirm that their definitions are accurately represented. Avoid repetition.
    For each term, cross-reference with the text to verify that the definition is accurate and contextually relevant. Utilize relevant knowledge to aid this process.
    Additionally, for definitions that reference other terms (like 'Union' or 'Confederacy'), ensure those referenced terms are also included in the list if not already present.
    Aim to refine the output into a perfectly concise and comprehensive list of terms and definitions, eliminating repetition and irrelevant content. 
    Your meticulous attention to detail is crucial for the integrity and utility of the final output.
    Response format:\nTerm: <term> Definition: <definition>

    Begin the final review and refinement process:"""
)


In [19]:
prompt_general = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "Focus on capturing a wide range of terms, ensuring no significant concept, name, or entity is overlooked. "
    "For each term, provide a clear and succinct definition that encapsulates its essence within the context. "
    "In cases where the definition of a term is not explicitly stated, use contextual clues to derive an appropriate meaning. "
    "This task is crucial for understanding the broader scope of the subject matter. "
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
)

prompt_detailed = (
    "Construct a detailed list of terms, particularly focusing on names, places, events, and other specific categories, along with their definitions. "
    "Ensure that each term, especially proper nouns and technical terms, is accompanied by a precise and comprehensive definition. "
    "Leverage the context to discern nuanced meanings and subtle distinctions between similar terms. "
    "This task is essential for a deep and accurate understanding of the specific elements within the subject matter. "
    "Maintain clarity in your list by following this structure for each entry:\nTerm: <term> Definition: <definition>"
)

In [20]:
megaprompt = (
     """Task: Conduct a thorough review of the document to create an exhaustive list of terms and their definitions. 
    Each term-definition pair should be listed on a separate line. 
    Focus on identifying a diverse range of terms, including proper nouns, technical terms, concepts, and significant entities relevant to the document's subject. 
    Provide clear, concise, and contextually derived definitions for each term. 
    Pay special attention to nuanced meanings and subtle distinctions, avoiding redundancy. 
    Only include variations of a term if they offer new and distinct information. 
    Exercise exceptional accuracy and thoroughness, using informed judgment in ambiguous cases. 
    The goal is to achieve a deep and accurate representation of the material, omitting extraneous details. 
    Response format:\nTerm: <term> Definition: <definition>"""

    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
)


prompt = (
    "completely understand what the material is about in terms of learning objective"
    "Find important information that would pertain to learning of the topic, those would be defined as terms"
    "Dont add any terms that take away from the learning objective and "
    "When sifting through the definitions and terms if you find very high similarities in definitions just combine the terms so that it can explain everything in one term"
)

In [21]:
bossPrompt = (
    """Task: Execute a meticulous analysis of the results from the following prompts. 
    First, rigorously examine each prompt's output, identifying all key terms along with their precise definitions. 
    Second, methodically compare the extracted terms across the prompts, paying close attention to their relevance and significance in relation to the overarching topic. 
    Next, selectively synthesize the information by choosing the most relevant terms from each prompt, focusing on their contribution to a cohesive understanding of the subject. 
    Finally, merge this knowledge to construct a comprehensive and integrated perspective, ensuring the synthesis is not only concise and accurate but also encapsulates the essence of the combined prompts.

    Initiate the detailed analysis and synthesis of the prompt results:"""
)


In [22]:
GBTPrompt = (
    """Task: Extract key terms and their definitions from the document. 
    For each term, especially proper nouns and technical terms, provide a clear definition. 
    Avoid redundancy; do not list variations of the same term. 
    Include notable names like 'Mathew Brady' and 'Abraham Lincoln' without fail. 
    Use context to infer meanings for ambiguous terms. 
    Desired Output Format:
    1. Term: <term>, Definition: <definition>
    Example:
    Term: Union, Definition: The northern states during the Civil War.
    Term: Abraham Lincoln, Definition: 16th President of the United States, led the nation during the Civil War.
    
    Begin extraction from the document below:"""
)

In [23]:
# Create a list of prompts
# Call the function with your index and prompts
d14= extract_terms_1(ds384, prompt2, 2)  # Here, 3 is the similarity_top_k value

In [24]:
d14

{'President Lincoln': 'The leader of the North during the Civil War who played a crucial role in keeping the nation together and made the decision to issue the Emancipation Proclamation.',
 'Emancipation Proclamation': 'A proclamation issued by President Lincoln that declared all slaves in Confederate territory to be free.',
 'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) from 1861 to 1865.',
 'Modern warfare': 'The harsh reality of warfare during the Civil War, which included high casualties and the use of new technologies such as photography to document the carnage.',
 'Gilded Age': 'The era following the Civil War characterized by social and political disillusionment, economic greed, and rapid industrialization in the North.',
 'Land Grant College Act': 'Legislation passed by Congress in 1862 that provided money from the sale of public lands to states for the establishment of universities focused on agriculture and mechanical art